# Barclays Analysis - Base, Barc, and Barc 2 Portfolio Iterations

## Import Libraries

In [1]:
import os
os.chdir('..')

import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary, util
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt
#,plots
from ipywidgets import interact, interact_manual
import pandas as pd
import plotly.graph_objects as go


## Import Equity Benchmark & Vortex Data

In [2]:
equity_bmk = 'SPTR'
strat_drop_list = ['99%/90% Put Spread','Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR',
       'GW Dispersion', 'Corr Hedge', 'Def Var']
include_fi = False
#create returns data dictionary for equity benchmark
returns= dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Import Barclays data

In [3]:
full_data= pd.read_excel(dm.NEW_DATA+'barclays.xlsx',
                                           sheet_name = 'data', index_col=0)

@interact
def display_performance():
    fig = go.Figure()
    for strat in full_data.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(full_data.index, format = '%m/%d/%Y'),
                                 y = full_data[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

interactive(children=(Output(),), _dom_classes=('widget-interact',))

## Compute Analytics (Return Stats and Hedge Metrics)

In [7]:
#create returns data dictionary for strategy
full_data_dict = dm.get_data_dict(full_data, data_type = 'returns')

#merge dictionaries
full_data_dict = dm.merge_dicts(returns,full_data_dict, False)

#compute analytics
analytics_freq_list = ['Weekly', 'Monthly']
full_data_analytics_dict = summary.get_analytics_data(full_data_dict)

## Display Analytics

In [8]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = full_data_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

## Compute Historical Selloffs

In [9]:
full_data_hist_dict = summary.get_hist_data(full_data_dict)

## Display Historical Selloffs

In [10]:
@interact
def display_selloffs():
    df_hist = full_data_hist_dict[False]
    return fmt.get_hist_styler(df_hist)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

## Create Report

In [12]:
strat_report = 'barc_analysis'
selloffs = True
rp.generate_strat_report(strat_report, full_data_dict, selloffs)

Computing Monthly Analytics...
Computing Weekly Analytics...
Computing Historical SellOffs...
"barc_analysis.xlsx" report generated in "C:\Users\RRQ1FYQ\Documents\RMP\EquityHedging\reports\" folder


# Equity Hedging Analysis
## Comparing with overal equity portfolio

## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [31]:
equity_bmk = 'SPTR'
include_fi = False
strat_drop_list = ['Vortex','99%/90% Put Spread']
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Add Barclays Data

In [32]:
new_strategy = full_data[['Barc2']]
new_strategy_dict = dm.get_data_dict(new_strategy, data_type = 'returns')
returns = dm.merge_dicts(returns, new_strategy_dict,False)
new_strat=True
weighted = [True, False]

## Get and display Notional Weights

In [33]:
notional_weights = [19,1,1.25,1,1,1,.25,1,.25]
df_weights = util.get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

,SPTR,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var,Barc2
Notional Weights (Billions),$19.00,$1.00,$1.25,$1.00,$1.00,$1.00,$0.25,$1.00,$0.25
Percentage Weights,100.00%,5.26%,6.58%,5.26%,5.26%,5.26%,1.32%,5.26%,1.32%
Strategy Weights,0.00%,14.81%,18.52%,14.81%,14.81%,14.81%,3.70%,14.81%,3.70%


## Compute Correlations

In [22]:
corr_freq_list = ['Weekly', 'Monthly']
corr_dict_1 = summary.get_corr_data(returns)

## Display Correlations

In [17]:
@interact
def display_correlations(frequency=corr_freq_list,
                         corr=['full', 'equity_down', 'equity_up'], plot_type=['heatmap', 'corrplot']):
    data = corr_dict_1[frequency]
    corr_df = data[False][corr][0]
    if plot_type == 'heatmap':
        return plots.draw_heatmap(corr_df, half=False)
    else:
        return plots.draw_corrplot(corr_df)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

## Compute Analytics (Return Stats and Hedge Metrics)

In [23]:
analytics_freq_list = ['Weekly', 'Monthly']
analytics_dict = summary.get_analytics_data(returns,analytics_freq_list,weighted,notional_weights,include_fi,new_strat)

## Display Analytics

In [24]:
@interact
def display_analytics(frequency=analytics_freq_list, weighted=weighted, stats=['return_stats', 'hedge_metrics']):
    data = analytics_dict[frequency]
    analytics = data[weighted]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

interactive(children=(Dropdown(description='frequency', options=('Weekly', 'Monthly'), value='Weekly'), Dropdo…

## Compute Historical Selloffs

In [25]:
hist_dict = summary.get_hist_data(returns,notional_weights=notional_weights, weighted=weighted)

## Display Historical Selloffs

In [ ]:
@interact
def display_selloffs(weighted=weighted):
    df_hist = hist_dict[weighted]
    return fmt.get_hist_styler(df_hist)

## Compute Quintile or Decile Analysis

In [ ]:
group_list=['Quintile','Decile']
@interact
def display_returns(group = group_list):
    quintile_df = summary.get_grouped_data(returns, group = group)
    return(quintile_df.style.format("{:.2%}"))

## Display Quintile or Decile Analysis

In [ ]:
@interact
def display_quintile_analysis(group = ['Quintile','Decile'], strat=list(returns['Monthly'].select_dtypes('float').columns)[1:]):    
    df = summary.get_grouped_data(returns, group = group)
    
    for col in df.columns:
        df[col] *= 100
        if not (col == equity_bmk or col ==strat):
            df.drop([col], axis=1, inplace=True)
    if group == "Quintile":
        title ='Quintile Analysis'
    else:
        title = 'Decile Analysis'
        
    fig = px.bar(df,barmode='group',title = title)
    fig.update_layout(
        title = {'text':"<b>{}</b>".format(title),'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(tickfont = dict(size=12)),
        yaxis = dict(ticksuffix="%"),
        legend_title="Strategies",
        plot_bgcolor='White'
    )
    fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>SPTR Monthly Returns Ranking</b>",
        title_font = {"size": 20},showline=False,linewidth=2,linecolor='black',mirror=False)
    fig.update_yaxes(title_font_family = "Calibri",title_text = "<b></b>",title_font = {"size": 20},
        showline=True,linewidth=1,linecolor='black',mirror=False)
    fig.show()

## Create Report

In [34]:
equity_hedge_report = 'equity_hedge_analysis_barclays_barc2'
selloffs = True
rp.get_equity_hedge_report(equity_hedge_report, returns, notional_weights, include_fi, new_strat, weighted[0], selloffs)

Computing Monthly Analytics...
Computing Weekly Analytics...
Computing Historical SellOffs...
"equity_hedge_analysis_barclays_barc2.xlsx" report generated in "C:\Users\RRQ1FYQ\Documents\RMP\EquityHedging\reports\" folder
